<a href="https://colab.research.google.com/github/bgkim/Byungki-Kim/blob/master/play%20music%20in%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import socket
import threading
import IPython
import portpicker

from six.moves import SimpleHTTPServer
from six.moves import socketserver

from google.colab import output

class _V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6


class _FileHandler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  """SimpleHTTPRequestHandler with a couple tweaks."""

  def translate_path(self, path):
    # Client specifies absolute paths.
    return path

  def log_message(self, fmt, *args):
    # Suppress logging since it's on the background. Any errors will be reported
    # via the handler.
    pass

  def end_headers(self):
    # Do not cache the response in the notebook, since it may be quite large.
    self.send_header('x-colab-notebook-cache-control', 'no-cache')
    SimpleHTTPServer.SimpleHTTPRequestHandler.end_headers(self)


def play_audio(filename):
  """Downloads the file to the user's local disk via a browser download action.

  Args:
    filename: Name of the file on disk to be downloaded.
  """

  started = threading.Event()
  port = portpicker.pick_unused_port()

  def server_entry():
    httpd = _V6Server(('::', port), _FileHandler)
    started.set()
    # Serve multiple requests, in case the audio is played more than once.
    httpd.serve_forever()

  thread = threading.Thread(target=server_entry)
  thread.start()
  started.wait()
  
  output.eval_js("""
    (()=> {
      const audio = document.createElement('audio');
      audio.controls = true;
      audio.autoplay = true;
      audio.src = `https://localhost:%(port)d%(path)s`;
      document.body.appendChild(audio);
    })()
  """% {'port': port, 'path': os.path.abspath(filename)})
print('playing')
play_audio('noise2.wav')